In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML: MLOps阶段5：部署：开始配置Vertex AI端点部署的自动缩放

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_autoscaling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_autoscaling.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage5/get_started_with_autoscaling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的MLOps生产。本教程涵盖阶段5：部署：使用自动扩展进行部署的入门。

### 目标

在本教程中，您将学习如何在部署`Model`资源到`Endpoint`资源时使用微调控制自动缩放配置。

本教程使用以下Google Cloud ML服务：

- `Vertex ML Prediction`

执行的步骤包括：

- 从TensorFlow Hub下载一个预训练的图像分类模型。
- 将预训练模型上传为`Model`资源。
- 创建一个`Endpoint`资源。
- 为不缩放（单节点）部署`Model`资源。
- 为手动缩放部署`Model`资源。
- 为自动缩放部署`Model`资源。
- 微调CPU利用率的缩放阈值。
- 微调GPU利用率的缩放阈值。
- 将CPU和GPU模型实例的混合部署与自动缩放到一个`Endpoint`资源中。

数据集

本教程使用的是来自TensorFlow Hub的一个预训练的图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 费用
本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- 云存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用量生成一个费用估算。

安装执行此笔记本所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-storage $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

### 重启内核

安装了额外的包之后，您需要重启笔记本内核以便它能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的 Google Cloud 项目

**无论您使用什么笔记本环境，下面的步骤都是必须的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您会获得一个可用于支付计算/存储成本的300美元的免费信用额度。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI、Compute Engine 和 Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component)。

4. 如果您正在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行单元格，确保
Cloud SDK 使用正确的项目来运行本笔记本中的所有命令。

**注意**：Jupyter以`!`为前缀运行的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本的其余操作。下面是 Vertex AI 支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不可以使用一个多区域存储桶来训练 Vertex AI。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加现场教程会话，您可能正在使用共享测试帐户或项目。为了避免在创建的资源上出现名称冲突，您需要为每个实例会话创建一个时间戳，并将该时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的谷歌云帐户

**如果您使用Vertex AI Workbench Notebooks**，则您的环境已经通过身份验证。跳过此步骤。

**如果您使用Colab**，请运行下面的单元格，并按提示通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务帐户**。

2. 在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

3. 在**向该服务帐户授予项目访问权限**部分，点击角色下拉列表。在过滤框中键入"Vertex AI"，选择**Vertex AI管理员**。在过滤框中键入"Storage Object Admin"，选择**Storage Object Admin**。

4. 点击创建。一个包含您密钥的JSON文件将下载到本地环境。

5. 在下面的单元格中将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用哪种笔记本环境，下面的步骤都是必需的。**

在初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源在不同会话中保留的地方。

请在下方设置您的云存储桶的名称。桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶尚未存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import tensorflow as tf
import tensorflow_hub as hub

### 初始化Vertex AI SDK for Python

为您的项目和相应的存储桶初始化Python版本的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机实例（VM）的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您应该指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

了解有关您所在地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

了解有关[机器类型的GPU兼容性](https://cloud.google.com/vertex-ai/docs/training/configure-compute#gpu-compatibility-table)的更多信息。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

设置预构建的容器

为预测设置预构建的Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

GPU_VERSION = "tf2-gpu.{}".format(TF)
CPU_VERSION = "tf2-cpu.{}".format(TF)

DEPLOY_IMAGE_GPU = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], GPU_VERSION
)

DEPLOY_IMAGE_CPU = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], CPU_VERSION
)

print("Deployment:", DEPLOY_IMAGE_GPU, DEPLOY_IMAGE_CPU, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置将用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

## 从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练模型，然后将其上传到`Vertex AI Model`资源。一旦您拥有了`Vertex AI Model`资源，该模型就可以部署到`Vertex AI Endpoint`资源。

### 下载预训练模型

首先，您可以从TensorFlow Hub下载预训练模型。该模型将作为TF.Keras层进行下载。为了完成模型，在本示例中，您将使用下载的TFHub模型作为层创建一个`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型文件

在这一步，模型已经在内存中。接下来，您可以将模型文件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

将TensorFlow Hub模型上传到`Vertex AI Model`资源

最后，您将来自TFHub模型和服务函数的模型文件上传到`Vertex AI Model`资源中。

*注意:* 当您将模型文件上传到`Vertex AI Model`资源时，需要指定相应的部署容器镜像。在本例中，您将使用仅支持CPU的部署容器。

In [ ]:
model = aiplatform.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE_CPU,
)

print(model)

创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。 至少，您需要为endpoint指定显示名称。 可选的，您可以指定项目和位置（区域）； 否则，设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的人类可读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）以键/值对形式定义`Endpoint`的用户自定义元数据。

此方法将返回一个`Endpoint`对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

将`Model`资源部署到一个`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个端点。每个被部署的`Vertex AI Model`资源都将拥有其自己的服务二进制的部署容器。

*注意:* 在前一个步骤中上传模型文件到`Vertex AI Model`资源时，您已经指定了TFHub模型的部署容器。

### 扩展

`Vertex AI Endpoint`资源支持三种类型的扩展:

- 无扩展: 服务二进制部署到单个VM实例。
- 手动扩展: 服务二进制部署到固定数量的多个VM实例。
- 自动扩展: 根据负载情况，服务二进制部署到不同数量的VM实例。

### 无扩展

在下一个例子中，您将把`Vertex AI Model`资源部署到一个`Vertex AI Endpoint`资源，没有任何扩展 -- 也就是说，单个VM (节点)实例。换句话说，当模型被部署时，将配置一个单个VM实例并保持配置，直到模型被取消部署。

在这个示例中，您将使用指定的最少参数部署模型，如下所示:

- `model`: 要部署的`Model`资源。
- `machine_type`: 每个VM实例的机器类型。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。

对于无扩展，单个VM实例在模型部署期间被配置。由于准备资源的要求，这可能需要几分钟。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

#### 显示缩放配置

一旦您的模型部署完成，您可以查询`Endpoint`资源，使用属性`endpoint.gca_resource.deployed_models`来检索已部署模型的缩放配置。

由于一个`Endpoint`资源可能有多个已部署模型，`deployed_models`属性会返回一个列表，每个已部署模型对应一个条目。在这个例子中，只有一个已部署模型，您可以通过`deployed_models[0]`来获取列表中的第一个条目，并显示属性`dedicated_resources`，该属性会返回机器类型和最小/最大节点数以进行缩放。对于不需要缩放的情况，最小/最大节点数将被设为一。

*注意：* 已部署模型标识符是指已部署模型的实例，而不是模型资源标识符。

In [ ]:
print(endpoint.gca_resource.deployed_models[0].dedicated_resources)

deployed_model_id = endpoint.gca_resource.deployed_models[0].id

解除模型

当您完成预测后，您可以从 `Endpoint` 资源中解除模型。这将取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy(deployed_model_id)

### 手动扩展

在下面的示例中，您将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源，用于手动扩展--一个固定数量（大于1）的VM实例。换句话说，当模型被部署时，固定数量的VM实例被预配并保持预配状态直到模型被取消部署。

在这个示例中，您以最小数量的指定参数部署模型，如下所示：

- `model`: 要部署的`Model`资源。
- `machine_type`: 每个VM实例的机器类型。
- `deployed_model_displayed_name`: 部署模型实例的人类可读名称。
- `min_replica_count`: 要预配的最小VM实例（节点）数量。
- `max_replica_count`: 要预配的最大VM实例（节点）数量。

对于手动扩展，固定数量的VM实例在模型部署期间被预配。

*注意:* 对于手动扩展，节点的最小和最大数量被设置为相同的值。

In [ ]:
MIN_NODES = MAX_NODES = 2

response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

显示缩放配置

在这个示例中，有一个单独部署的模型，您可以通过列表中的第一个条目`deployed_models[0]`检索到缩放配置。然后显示`dedicated_resources`属性，这将返回机器类型和最小/最大节点数量以进行缩放。对于手动缩放，最小/最大节点将被设置为相同的值，大于1。

In [ ]:
print(endpoint.gca_resource.deployed_models[0].dedicated_resources)

deployed_model_id = endpoint.gca_resource.deployed_models[0].id

#### 卸载模型

当您完成预测时，您可以从`Endpoint`资源中卸载模型。这将取消所有计算资源并停止为部署模型计费。

In [ ]:
endpoint.undeploy(deployed_model_id)

### 自动缩放

在下一个示例中，您将将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源以进行自动缩放，即变量数量（大于1）的VM实例。换句话说，当模型部署时，将提供最小数量的VM实例。随着负载的变化，提供的实例数量可能动态增加到最大VM实例数量，并减少到最小数量的VM实例。提供的VM实例数量永远不会少于最小值或多于最大值。

在此示例中，您将使用指定参数的最少量部署模型，如下所示：

- `model`：要部署的`Model`资源。
- `machine_type`：每个VM实例的机器类型。
- `deployed_model_displayed_name`：部署模型实例的人类可读名称。
- `min_replica_count`：要提供的VM实例（节点）的最小数量。
- `max_replica_count`：要提供的VM实例（节点）的最大数量。

对于自动缩放，在部署模型时将提供最小数量的VM实例。

*注意：*对于自动缩放，最小节点数必须设置为大于零的值。换句话说，至少会有一个VM实例被提供。

In [ ]:
MIN_NODES = 1
MAX_NODES = 2

response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

#### 显示缩放配置

在这个例子中，有一个已部署的模型，并且您可以将缩放配置作为列表中的第一个条目进行检索：`deployed_models[0]`。然后，您可以显示`dedicated_resources`属性，该属性将返回机器类型和缩放的最小/最大节点数。对于自动缩放，最大节点数将被设置为大于最小节点数的值。

In [ ]:
print(endpoint.gca_resource.deployed_models[0].dedicated_resources)

deployed_model_id = endpoint.gca_resource.deployed_models[0].id

#### 卸载模型

当您完成预测时，您可以从 `Endpoint` 资源中卸载模型。这将取消所有计算资源并停止部署模型的计费。

In [ ]:
endpoint.undeploy(deployed_model_id)

### 设置缩放阈值

`Endpoint`资源支持基于两个指标的自动缩放: CPU利用率和GPU工作循环。这两个指标通过计算每个部署模型的平均利用率来衡量。一旦利用率指标超过某个阈值一段时间，VM实例（节点）的数量会相应地调整上升或下降。


#### CPU阈值

在先前的示例中，部署的VM实例仅带有CPU -- 即没有硬件加速器。默认情况下（自动缩放中），CPU利用率指标设置为60%。在部署模型时，可以指定参数`autoscaling_target_cpu_utilization`来设置非默认值。

In [ ]:
MIN_NODES = 1
MAX_NODES = 4

response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    autoscaling_target_cpu_utilization=50,
)

显示缩放配置

在这个例子中，有一个部署的模型，您可以将缩放配置作为列表中的第一个条目检索出来：`deployed_models[0]`。然后，您可以显示`dedicated_resources`属性，它将返回机器类型和可扩展的节点的最小/最大数量，以及CPU利用率的目标值：`autoscaling_metric_specs`。

In [ ]:
print(endpoint.gca_resource.deployed_models[0].dedicated_resources)

deployed_model_id = endpoint.gca_resource.deployed_models[0].id

取消模型部署

当你完成预测时，你可以从`Endpoint`资源中取消模型部署。这将取消所有计算资源并停止为部署的模型计费。

In [ ]:
endpoint.undeploy(deployed_model_id)

### 上传适用于 GPU 部署图像的 TensorFlow Hub 模型

接下来，您可以上传第二个实例的 TensorFlow Hub 模型作为`Model`资源 -- 但是相应的服务容器支持 GPU。

In [ ]:
model_gpu = aiplatform.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE_GPU,
)

print(model)

GPU阈值

在此示例中，部署的 VM 实例被配置为使用硬件加速器（即 GPUs），通过指定以下参数：

- `accelerator_type`: 硬件（例如 GPU）加速器的类型。
- `accelerator_count`: 每个预置 VM 实例的硬件加速器数量。

支持的 GPU 类型和数量特定于机器类型和区域。

了解更多关于[每个机器类型的 GPU 类型和数量](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute)。

了解更多关于[每个区域可用的 GPU 类型](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

默认情况下（在自动扩展中），GPU 利用率指标设置为 60%。在部署模型时，指定参数 `autoscaling_target_accelerator_duty_cycle` 来设置非默认值。

在提供服务时，如果 CPU 利用率或 GPU 负载周期超出或低于某个阈值一段时间，那么将触发自动扩展。

In [ ]:
MIN_NODES = 1
MAX_NODES = 2

response = endpoint.deploy(
    model=model_gpu,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU.name,
    accelerator_count=DEPLOY_NGPU,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    autoscaling_target_accelerator_duty_cycle=50,
)

#### 显示缩放配置

在此示例中，有一个已部署的模型，您可以将缩放配置作为列表中的第一个条目检索：`deployed_models[0]`。然后显示`dedicated_resources`属性，该属性将返回机器类型和可缩放的节点的最小/最大数量，以及GPU工作周期的目标值：`autoscaling_metric_specs`。

In [ ]:
print(endpoint.gca_resource.deployed_models[0].dedicated_resources)

deployed_model_id = endpoint.gca_resource.deployed_models[0].id

### 将多个模型部署到“Endpoint”资源

接下来，您将两个模型部署到同一个“Endpoint”资源，并在它们之间分配预测请求流量。一个模型将使用GPU，占流量的80%，另一个将使用CPU，占流量的20%。

您已经将GPU版本的模型部署到“Endpoint”资源中。在这个例子中，您将向同一个“Endpoint”资源添加第二个模型实例 - CPU版本，并指定模型之间的流量分配。在这个例子中，`traffic_split`参数被指定如下：

- `"0": 20`：被部署的模型（默认ID为0）将接收20%的流量。
- `deployed_model_id: 80`：现有部署的模型（通过其部署的模型ID指定）将接收80%的流量。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    autoscaling_target_cpu_utilization=50,
    traffic_split={"0": 20, deployed_model_id: 80},
)

#### 显示缩放配置

在这个例子中，有两个部署模型，分别是CPU版本和GPU版本。

In [ ]:
print(endpoint.gca_resource.deployed_models)

解除部署模型

当您完成预测工作后，您将从“端点”资源中解除部署所有模型。这将取消所有计算资源，并停止对已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

#### 删除模型实例

方法'delete（）'将删除模型。

In [ ]:
model.delete()
model_gpu.delete()

删除端点

方法'delete()'会删除端点。

In [ ]:
endpoint.delete()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = True

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI